In [ ]:
%matplotlib inline
import matplotlib

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import discopt
import pandas as pd
import numpy as np
import copy
import sklearn
import sklearn.datasets
import xgboost as xgb
import scipy
from scipy import stats

In [ ]:
n_jobs = 3
ntrials = 20

# Prepare data 

In [ ]:
np.random.seed(0)
digits = sklearn.datasets.load_digits()
x = digits.data
y = digits.target

ntrain = 100
order = np.random.permutation(y.size)
tr = order[:ntrain]
val = order[ntrain:]
ytr = y[tr]
xtr = x[tr, :]
yval = y[val]
xval = x[val, :]


# DISCOPT SHORTCUT FOR XGB 

In [ ]:
# parameter space is proposed in discopt.py file

In [ ]:
# write your train_and_valid function
def train_and_valid(clf):
    # put here trainig and validation, return objective
    # optimizer would try to minimize this value, so use NEGATIVE accuracy/AUC/F1...
    # validation method is up to you
    # clf is xgboost.sklearn.XGBoostClassifier instance
    clf.fit(xtr, ytr)
    pred = clf.predict(xval)
    obj = -(pred == yval).mean() # (minus) accuracy
    return obj

In [ ]:
best_params, scores = discopt.search_params_for_xgb(train_valid_func=train_and_valid,
                                                    train_data_shape=xtr.shape,
                                                    ntrials=ntrials,
                                                    n_jobs=n_jobs)

In [ ]:
# the long 'stagnations' of 'best value so far' is typical for every optimizer, at least for this particular problem

In [ ]:
best_params

In [ ]:
# dataframe with objective and 
scores.head()